In [ ]:
def make_date(year,month):
    date = ''
    if month < 10:
        date = str(year) + '0' + str(month)
    else: 
        date = str(year) + str(month)
    return date

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import sys
import pprint

def flatten(l):
    flatList = []
    for elem in l:
        # if an element of a list is a list
        # iterate over this list and add elements to flatList 
        if type(elem) == list:
            for e in elem:
                flatList.append(e)
        else:
            flatList.append(elem)
    return flatList

def comment_crawler(url):
    oid=url.split("oid=")[1].split("&")[0]
    aid=url.split("aid=")[1]
    page=1  
    header = {
    "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
    "referer":url,
    } 

    
    while True :
        c_url="https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news"+oid+"%2C"+aid+"&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page="+str(page)+"&refresh=false&sort=FAVORITE" 
        # 파싱하는 단계입니다.
        r=requests.get(c_url,headers=header)
        cont=BeautifulSoup(r.content,"html.parser")    
        
        total_comm=str(cont).split('comment":')[1].split(",")[0]

        match=re.findall('"contents":([^\*]*),"userIdNo"', str(cont))
        # 댓글을 리스트에 중첩합니다.
        List.append(match)
        # 한번에 댓글이 20개씩 보이기 때문에 한 페이지씩 몽땅 댓글을 긁어 옵니다.
        if int(total_comm) <= ((page) * 20):
            break
        else : 
            page+=1
            
    flatten(List)
    
    return List



In [ ]:
import pandas as pd
import html

read_path = '../article/'
write_path = '../comment/'

column_name = ['date','co','headline','text','url']

for year in range(2018,2019):
    for month in range(3,4):
        date = make_date(year,month)
        article = pd.read_csv(f'{read_path}Article_부동산_{date}_{date}.csv', names = column_name, encoding = 'cp949')
        
        df = pd.DataFrame({
            "url" : article['url'],
            "comment" : [comment_crawler(url) for url in article['url']]
        })
        
        pd.to_csv(f'{write_path}Article_부동산_{date}_{date}_댓글.csv')